In [1]:
import arcpy.management
import requests
import zipfile
import os
import tempfile
import datetime
import time
import arcpy
import hashlib
import arcpy


# Functions
<font color="red">**CAUTION**: Modify the functions below only if you understand the consequences.</font>

In [7]:

def download_and_extract(year):
    # Download the zipped file
    url = f"https://sentinel.esa.int/documents/d/sentinel/sentinel-2a-acquisition-plans-{year}"
    response = requests.get(url)

    # Save the zipped file to a temporary directory
    temp_dir = tempfile.mkdtemp()
    zip_path = os.path.join(temp_dir, "sentinel.zip")
    with open(zip_path, "wb") as f:
        f.write(response.content)

    # Extract the zipped file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    return temp_dir
def calculate_size(cell_size, max_pixels):
    # Calculate the size in square meters
    size_sqm = (cell_size * max_pixels)**2

    # Convert the size to square kilometers
    size_sqkm = size_sqm / 1e6

    return size_sqkm
def create_tessellation(aoi, cell_size, max_pixels):

    # Create the tessellation
    tessellation = arcpy.GenerateTessellation_management(
        Output_Feature_Class="tessellation",
        Extent=aoi,
        Shape_Type="SQUARE",
        Size =  f"{calculate_size(cell_size, max_pixels)} SquareKilometers"

    )
    # return tessellation
    # Initialize the list of extents
    extents = []

    # Assume tessellation is the output from the create_tessellation function
    with arcpy.da.SearchCursor("tessellation", ["SHAPE@"]) as cursor:
        for row in cursor:
            feature = row[0]
            extent = feature.extent

            # Add the extent to the list
            extents.append(f'{extent.XMin},{extent.YMin},{extent.XMax},{extent.YMax}')

    return extents, tessellation


def parse_kml_files(directory, start_date, end_date, aoi):
    # Initialize the list of acquisition dates
    acq_dates = []
    kmlgdbs = []
    # Loop through the files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".kml"):
            # Extract the acquisition dates from the KML file name
            acq_start, acq_end = filename.split("_")[5:7]
            acq_start = datetime.datetime.strptime(acq_start, "%Y%m%dT%H%M%S")
            acq_end = datetime.datetime.strptime(acq_end.split(".")[0], "%Y%m%dT%H%M%S")

            # Check if the acquisition dates fall within the defined start and end dates
            if start_date <= acq_start <= end_date or start_date <= acq_end <= end_date:
                # Convert the KML to a Layer
                arcpy.env.workspace = directory

                arcpy.conversion.KMLToLayer(os.path.join(directory, filename), directory)

                # Loop through all the file geodatabases in the workspace
                gdb = [gdb for gdb in arcpy.ListWorkspaces('*', 'FileGDB') if gdb not in kmlgdbs][0]
                kmlgdbs.append(gdb)

                arcpy.env.workspace = gdb

                # Find the polygon feature class in the file geodatabase
                
                datasets = arcpy.ListDatasets(feature_type='feature')
                datasets = [''] + datasets if datasets is not None else []

                for ds in datasets:
                    for fc in arcpy.ListFeatureClasses(feature_dataset=ds):
                        if arcpy.Describe(fc).shapeType == "Polygon":
                            # Perform a spatial join with the input AOI
                            arcpy.analysis.SpatialJoin(
                                target_features=f"{ds}\\{fc}",
                                join_features=aoi,
                                out_feature_class="Join",
                                join_operation="JOIN_ONE_TO_ONE",
                                join_type="KEEP_ALL",
                                field_mapping=r'Name "Name" true true false 320 Text 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,Name,0,319;FolderPath "FolderPath" true true false 320 Text 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,FolderPath,0,319;SymbolID "SymbolID" true true false 4 Long 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,SymbolID,-1,-1;AltMode "AltMode" true true false 2 Short 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,AltMode,-1,-1;Base "Base" true true false 8 Double 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,Base,-1,-1;Clamped "Clamped" true true false 2 Short 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,Clamped,-1,-1;Extruded "Extruded" true true false 2 Short 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,Extruded,-1,-1;TimeSpan "TimeSpan" true true false 2 Short 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,TimeSpan,-1,-1;TimeStamp "TimeStamp" true true false 2 Short 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,TimeStamp,-1,-1;BeginTime "BeginTime" true true false 255 Text 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,BeginTime,0,254;EndTime "EndTime" true true false 255 Text 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,EndTime,0,254;Snippet "Snippet" true true false 268435455 Text 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,Snippet,0,268435454;PopupInfo "PopupInfo" true true false 268435455 Text 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,PopupInfo,0,268435454;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,S2 Planned Observations from 20230112T120000 to 20230130T150000\Polygons,Shape_Area,-1,-1;Shape_Length_1 "Shape_Length" false true true 8 Double 0 0,First,#,Cantabria_Boundaries_WGS1984_Pro,Shape_Length,-1,-1;Shape_Area_1 "Shape_Area" false true true 8 Double 0 0,First,#,Cantabria_Boundaries_WGS1984_Pro,Shape_Area,-1,-1',
                                match_option="INTERSECT",
                                search_radius=None,
                                distance_field_name="",
                                match_fields=None
                            )
                            # Open a search cursor on the in_memory output
                            with arcpy.da.SearchCursor("join", ["BeginTime", "EndTime"], "Join_Count = 1") as cursor:
                                for row in cursor:
                                    date = row[0].split("T")[0]
                                    start_of_day_unix = int(time.mktime(datetime.datetime.strptime(date, '%Y-%m-%d').timetuple())) * 1000
                                    end_of_day_unix = start_of_day_unix + 86399999
                                    # Append the start date of the filtered features to the acq_dates list
                                    acq_dates.append(f"{start_of_day_unix}, {end_of_day_unix}")
                                del cursor, row
                        arcpy.management.Delete("join")
                            
                arcpy.env.workspace = directory

    return acq_dates

def generate_token(username, password, referer):
    # Define the URL for generating a token
    url = "https://www.arcgis.com/sharing/rest/generateToken"

    # Define the parameters for generating a token
    params = {
        "username": username,
        "password": password,
        "referer": referer,
        "f": "json"
    }

    # Send a POST request to generate a token
    response = requests.post(url, params)

    # Extract the token from the response
    token = response.json()["token"]

    return token

def download_image(token, acq_date, extent, tessellation, output_dir, username, password):
    arcpy.env.overwriteOutput = True

    arcpy.SignInToPortal("https://www.arcgis.com", 
                     username, password)
    # Define the URL for exporting an image
    url = "https://sentinel.arcgis.com/arcgis/rest/services/Sentinel2/ImageServer"
    # Convert the acquisition dates from Unix timestamps to the ddMMYYYY format
    acq_start, acq_end = acq_date.split(", ")
    acq_start = datetime.datetime.fromtimestamp(int(acq_start) / 1000).strftime("%d%m%Y")
    acq_end = datetime.datetime.fromtimestamp(int(acq_end) / 1000).strftime("%d%m%Y")

    arcpy.management.MakeImageServerLayer(in_image_service=url, out_imageserver_layer="sentinel2_service", processing_template="None")
    arcpy.management.SelectLayerByLocation(in_layer="sentinel2_service", overlap_type="INTERSECT", select_features=tessellation)
    arcpy.management.SelectLayerByAttribut(in_layer_or_view="sentinel2_service", selection_type="NEW_SELECTION",
                                           where_clause=f"acquisitiondate >= timestamp '{acq_start}' And acquisitiondate < timestamp '{acq_start + timedelta(days=1)}' And cloudcover < 0.3")
    

    acq_start, acq_end = acq_date.split(", ")
    # Generate a unique hash for the extent
    extent_hash = hashlib.md5(str(extent).encode()).hexdigest()

    with arcpy.EnvManager(extent=f'{extent} PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]'):
        
        arcpy.management.SplitRaster(
        in_raster="sentinel2_service",
        out_folder=output_dir,
        out_base_name=f"sentinel2_{acq_start}_{acq_end}_{extent_hash}",
        split_method="SIZE_OF_TILE",
        format="TIFF",
        resampling_type="NEAREST",
        num_rasters="1 1",
        tile_size="2048 2048",
        overlap=0,
        units="PIXELS",
        cell_size=None,
        origin=None,
        split_polygon_feature_class=None,
        clip_type="NONE",
        template_extent="DEFAULT",
        nodata_value=""
    )
def process_raster_data(output_dir):
    with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]'):
        outgdb = os.path.join(output_dir, "sentinel2.gdb")
        arcpy.management.CreateFileGDB(output_dir, "sentinel2.gdb")

        arcpy.management.CreateMosaicDataset(
            in_workspace=outgdb,
            in_mosaicdataset_name="sentinel2",
            coordinate_system='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
            num_bands=None,
            pixel_type="",
            product_definition="",
            product_band_definitions=None
        )
        arcpy.management.AddRastersToMosaicDataset(
        in_mosaic_dataset=os.path.join(outgdb, "sentinel2"),
        raster_type="Raster Dataset",
        input_path=f"{output_dir}",
        update_cellsize_ranges="UPDATE_CELL_SIZES",
        update_boundary="UPDATE_BOUNDARY",
        update_overviews="UPDATE_OVERVIEWS",
        maximum_pyramid_levels=None,
        maximum_cell_size=0,
        minimum_dimension=1500,
        spatial_reference=None,
        filter="",
        sub_folder="SUBFOLDERS",
        duplicate_items_action="ALLOW_DUPLICATES",
        build_pyramids="BUILD_PYRAMIDS",
        calculate_statistics="CALCULATE_STATISTICS",
        build_thumbnails="BUILD_THUMBNAILS",
        operation_description="",
        force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
        estimate_statistics="ESTIMATE_STATISTICS",
        aux_inputs=None,
        enable_pixel_cache="NO_PIXEL_CACHE",
    )
        
    arcpy.management.MakeRasterLayer(os.path.join(outgdb, "sentinel2"), "sentinel2")

    # Define the feature class
    fc = "sentinel2"

    # Add the "AquisitionDate" field
    arcpy.AddField_management(fc, "AquisitionDate", "DATE", field_alias="Aquisition Date")

    # Define the update cursor
    cursor = arcpy.da.UpdateCursor(fc, ["Name", "AquisitionDate"])
    for row in cursor:
        if "Sen" in row[0]:
            # Extract the date string from the field value
            date_str = row[0].split("_")[1]

            # Convert the date string to a date object
            date_obj = datetime.datetime.strptime(date_str, "%d%m%Y").date()

            # Update the "AquisitionDate" field
            row[1] = date_obj

            # Update the row
            cursor.updateRow(row)
    del cursor, row
    return os.path.join(outgdb, "sentinel2")


# Variables

In [4]:

# Define Area of Interest (AOI) and output directory
# Change the aoi to the path of the feature class or shapefile that represents the area of interest
aoi = r"D:\OneDrive - Esri\Demos & Blogs\ArcGIS Resources\GeoAi & Deep Learning\Demos\Biomass_Cantabria\Biomass_Cantabria.gdb\aoi"
# Change the output_dir to the directory where you want to save the downloaded images and the processed raster data
output_dir = r"C:\Users\rami8629\AppData\Local\Temp\ArcGISProTemp7392\Untitled\sentinel2"

#Input AGOL credentials
username = input("Enter your ArcGIS Online username: ")
password = input("Enter your ArcGIS Online password: ")


In [5]:

# Define the start and end dates
start_date = datetime.datetime(2023, 5, 1)
end_date = datetime.datetime(2023, 5, 30)


In [8]:

years = list(range(start_date.year, end_date.year+1))

for year in years:
    temp_dir = download_and_extract(year)

acq_dates = parse_kml_files(temp_dir, start_date, end_date, aoi)
acq_dates = list(set(acq_dates))

extents, tessellation = create_tessellation(aoi, 10, 2048)

# Generate a token
token = generate_token(username, password, "https://www.arcgis.com")


# Loop through the list of acquisition dates and extents
for acq_date in acq_dates:
    for extent in extents:
        download_image(token, acq_date, extent, tessellation, output_dir, username, password)

# Process the raster data
out_mosaic = process_raster_data(output_dir)

print(f"Processed raster data saved to {out_mosaic}")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000368: Invalid input data.
Failed to execute (SelectLayerByLocation).


In [1]:
from arcgis.gis import GIS
from arcgis.raster import ImageryLayer
from arcgis.raster.functions import apply
import hashlib
import datetime

output_dir = r"E:\OneDrive - Esri\Demos & Blogs\ArcGIS Resources\GeoAi & Deep Learning\Demos\Biomass_Cantabria\Sentinel2_15Bands"
username = "ralouta.aiddev"
password = input("Enter your ArcGIS Online password: ")
# Sign in to ArcGIS Online
gis = GIS("https://www.arcgis.com", username, password)


C:\Users\rami8629\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [13]:

sentinel_item=gis.content.search('255af1ceee844d6da8ef8440c8f90d00', 'Imagery Layer', outside_org=True)[0]
s2_layer = sentinel_item.layers[0]
sentinel_AllBands = apply(s2_layer, 'None')
# acq_date = f'{1685484000000}, {int(1688076000000+(43200000*1.5))}'
acq_date = '1685484000000, 1685548800000'
extent = '-523457.48640000075,5350736.778700002,-520897.48640000075,5353296.778700002'
# Convert the acquisition dates from Unix timestamps to the ddMMYYYY format
acq_start, acq_end = acq_date.split(", ")
acq_start = datetime.datetime.fromtimestamp(int(acq_start) / 1000).strftime("%d%m%Y")
acq_end = datetime.datetime.fromtimestamp(int(acq_end) / 1000).strftime("%d%m%Y")

# Generate a unique hash for the extent
extent_hash = hashlib.md5(str(extent).encode()).hexdigest()

# Define the output filename
out_file = f"sentinel2_{acq_start}_{acq_end}_{extent_hash}.tif"

# Export the image
image = sentinel_AllBands.export_image(bbox=extent, 
                     save_folder=output_dir, 
                     save_file=out_file, 
                     image_sr=3857, 
                     bbox_sr=3857, 
                     size=[250, 250],
                     time=acq_date,
                     f="image", 
                     export_format="tiff")

In [12]:
acq_date

'1688076000000, 1688140800000'